In [2]:
%load_ext autoreload
%autoreload 1
%aimport ecg_get_data
%aimport Models
%aimport train_test_validat
%aimport self_attention
%aimport ECGplot
import Models 
from train_test_validat import *
from self_attention import *
import  ecg_get_data 
import ECGplot

import torch
import torch.utils.data as Data
import matplotlib.pyplot as plt

/opt/conda/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
data_path =  './npy_ECG/' #路径
lable_path = './label.npy'
test_model_path = "./model/20220908_133803/best_model_0.pt"
lead_index = ['I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6']
EcgChannles_num = 12
EcgLength_num = 5000
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(DEVICE)
test_model = torch.load(test_model_path).to(DEVICE)



In [ ]:
x = load_data(data_path,EcgChannles_num=EcgChannles_num,EcgLength_num=EcgLength_num)
y = load_label(lable_path)

In [ ]:
def MaxMinNormalization(x,Max,Min):
    x = (x - Min) / (Max - Min);
    return x

In [ ]:
test_x = x
test_y = y
test_x_norm = MAX_MIN_normalization_by_feactures(test_x)
test_x_norm = torch.FloatTensor(test_x_norm)  #turn numpy to tensor
test_y_norm = torch.LongTensor(test_y)
test_dataset = Data.TensorDataset(test_x_norm, test_y_norm)
test_dataloader = Data.DataLoader(dataset=test_dataset, batch_size=1, shuffle=False)
test_model.eval()
acclist = []
errlist = []
attention_value_timestep = np.zeros((len(x),312))
for i,data in enumerate(test_dataloader,0):
    inputs,labels = data[0].to(DEVICE),data[1].to(DEVICE)
    outputs = test_model(inputs)
    #print(labels)
    _,pred = outputs.max(1) # 求概率最大值对应的标签
    print("the label :{labels},pred is {pred}".format(labels=labels[0],pred=pred[0]))
    attention_value_timestep[i] = (((test_model.attention_value2.to('cpu'))[0]).detach().numpy()).sum(axis=0) #将得到的attention值(5000,5000)每行叠加起来，得到（5000，）的attention值
    attention_value_timestep[i] = MaxMinNormalization(attention_value_timestep[i],attention_value_timestep[i].max(),attention_value_timestep[i].min())
    if(pred == labels):
        acclist.append(i) 
    else:
        errlist.append(i)
    
#attention_value_lead = MaxMinNormalization(attention_value_lead,attention_value_lead.max(),attention_value_lead.min())

In [ ]:
errlist.__len__()

In [ ]:
attention_value__each_timestep = np.zeros(EcgLength_num,)
factor = int(EcgLength_num/len(attention_value_timestep[0]))
for i in range(0,EcgLength_num,factor):
    if (i/factor)>=len(attention_value_timestep[0]):
        break
    attention_value__each_timestep[i:i+factor] = attention_value_timestep[0][int(i/factor)]
x_index = np.arange(0,EcgLength_num)
fig, axs = plt.subplots(nrows=12, ncols=1, sharex=True,sharey=True,figsize=(90,40), constrained_layout=True)
for i,ax in enumerate(axs.flat):
    #plot_y = x[1,i,:]*(4.88)
    plot_y = np.array(test_x[0,i,:])*4.88
    ECGplot.plot_multicolored_line(fig,ax,x = x_index,y= plot_y,color_depend=attention_value__each_timestep,cmap="turbo",y_name = str(lead_index[i])+" Voltage(mV)"
                                        ,title = lead_index[i])

In [3]:
from pathlib import Path
from tqdm import tqdm
file_path = Path("./xml/xml")
file_list = list(file_path.glob('*.xml')) # list(images_path.glob('*.png'))
file_list =[str(x) for x in file_list]
file_list.sort(key=lambda x:int(x.split('/')[-1].split('_')[0])) #按“/”分割，取最后一个，并把最后后一个按'_'分割，
                                                                 #按'_'分割后再取第0个，即为编号，按此排序
for file in tqdm(file_list):
    ecg = ecg_get_data.get_ECG_form_xml(file,12,5000)
    np.save('./npy_ECG_int/'+(file.split('/')[-1].split('.')[0])+".npy",ecg)

100%|██████████| 26782/26782 [1:51:23<00:00,  4.01it/s]


In [ ]:
file_path = Path("./xml/NHTN/")
file_list = list(file_path.glob('*.xml')) # list(images_path.glob('*.png'))
file_list =[str(x) for x in file_list]
file_list.sort(key=lambda x:int(x.split('/')[-1].split('_')[0])) #按“/”分割，取最后一个，并把最后后一个按'_'分割，
                                                                 #按'_'分割后再取第0个，即为编号，按此排序
for file in tqdm(file_list):
    ecg = ecg_get_data.get_ECG_form_xml(file,12,5000)
    np.save('./npy_ECG_int/'+(file.split('/')[-1].split('.')[0])+".npy",ecg)